# This file creates images from the videos to finetune the VGG architecture

In [1]:
import os
import glob
import pylab
import imageio
import pandas as pd
import random 
from PIL import Image
import PIL
import cv2
import matplotlib.pyplot as plt
from skimage.io import imsave
import shutil
import IPython
import json
import time
import numpy as np
import re

In [2]:
from google.colab import drive
drive.mount('/content/drive')

general_info_all = pd.read_csv("/content/drive/Shareddrives/CV_videos/" + "other_docs/SVW.csv", sep = ";")[["FileName", "# frames"]]

Mounted at /content/drive


# Get some random frames from video

This code here gets some random frames from all processed videos to later train the pretrained CNN on it

In [3]:
path_to_sports = "/content/drive/Shareddrives/CV_videos/Video_Processed/"
path_to_images_from_video = "/content/drive/Shareddrives/CV_videos/Images_from_videos/"
general_info_all = pd.read_csv("/content/drive/Shareddrives/CV_videos/" + "other_docs/SVW.csv", sep = ";")[["FileName", "# frames"]]
frames_per_video_needed = 20

In [ ]:
# Helper function: gets the frames and saves them 
def produce_more_frames(video_name: str, path_to_folder_containing_video: str, path_to_folder_containing_images: str, nr_frames_from_video_existing: int, frames_per_video_needed: int, general_info_all):
    try:
        path_to_file = (os.path.join(path_to_folder_containing_video, video_name)) + str(".mp4")
        vid = imageio.get_reader(path_to_file,  'ffmpeg')
        trv = (general_info_all[general_info_all["FileName"] == (video_name + ".mp4")]["# frames"]).item()
        while nr_frames_from_video_existing < frames_per_video_needed:
            frame_to_save = random.randint(0, trv)
            image_to_save = vid.get_data(frame_to_save)
            name_new_image = os.path.join(path_to_folder_containing_images, video_name) + "_" + str(nr_frames_from_video_existing) + ".png"
            imsave(name_new_image, image_to_save)
            nr_frames_from_video_existing += 1
    except IOError:
        pass

In [ ]:
not_done_yet = True
while not_done_yet:
    try:
        for folder_sport in os.listdir(path_to_sports):
            if not os.path.isdir(os.path.join(path_to_images_from_video, folder_sport)):
                os.mkdir(os.path.join(path_to_images_from_video, folder_sport))

            print("Doing " + str(folder_sport) + "...")
            for video in os.listdir(os.path.join(path_to_sports, folder_sport)):
                video_name = video.strip(".mp4")
                path_to_dir_sport = os.path.join(path_to_images_from_video, folder_sport)
                nr_frames_from_video_existing = len(glob.glob1(path_to_dir_sport, str(video_name) + "*"))

                if nr_frames_from_video_existing < frames_per_video_needed:
                    print(video_name)
                    produce_more_frames(video_name, os.path.join(path_to_sports, folder_sport), os.path.join(path_to_images_from_video, folder_sport), nr_frames_from_video_existing, frames_per_video_needed, general_info_all)
                else:
                    pass
        
        not_done_yet = False
    except Exception as e:
        print("Restarting this cell")
        time.sleep(3) # wait for 10 seconds before restarting the cell
        IPython.display.display(IPython.display.HTML("<script>Jupyter.notebook.execute_cells_below()</script>"))

print("Done!")

Doing baseball...
Doing archery...
Doing gymnastics...
3076___94cdf4eff8064826a0b8396b6d19220c
Doing basketball...
Doing bowling...
Doing discusthrow...
2002___d420c17598064bc2b7b05e85ff55651f
Doing golf...
Doing hammerthrow...
Doing highjump...
Doing javelin...
333___d96496b6dd484b449aa3fc8e15050168
Doing longjump...
1573___c0f7b9c5a78d4976ad014ede1d259f1b
Doing polevault...
Doing running...
8578___40b0f6b4bd65401da67f38d70c2ab64f
Doing shotput...
20862___6a593152d51e4946961061de0fb3ce6d


# This file creates images cleaned json keypoints to finetune VGG

In [ ]:
#!mkdir /content/drive/Shareddrives/CV_videos/Images_from_json

In [4]:
path_to_sports = "/content/drive/Shareddrives/CV_videos/Video_Processed/"
path_to_clean_json = "/content/drive/Shareddrives/CV_videos/Cleaned_json"
path_to_images_from_json = "/content/drive/Shareddrives/CV_videos/Images_from_json"
general_info_all = pd.read_csv("/content/drive/Shareddrives/CV_videos/" + "other_docs/SVW.csv", sep = ";")[["FileName", "# frames"]]
frames_per_clean_json_needed = 7

In [ ]:
# Helper function: reconstructs posture and saves it 
def create_posture_image_from_json(clean_file_name: str, path_to_folder_containing_json: str, path_to_folder_containing_images: str, nr_frames_from_json_existing: int, frames_per_json_needed: int, general_info_all):
    connections = [(0, 1), (0, 2), (2, 4), (1, 3), (0, 5), (0, 6), (5, 7), (6, 8), (7, 9), (8, 10), (0, 11), (0, 12), (11, 13), (12, 14), (13, 15), (14, 16)]

    try:
        path_to_file = (os.path.join(path_to_folder_containing_json, clean_file_name)) + str(".json")
        
        ## Here open Json
        f = open(path_to_file)
        json_data = json.load(f)

        ## get number of keypoints 
        trv = len(json_data)

        #print("length data:", trv)
        W = general_info_all[general_info_all["FileName"] == clean_file_name + ".mp4"]["W"].item()
        H = general_info_all[general_info_all["FileName"] == clean_file_name + ".mp4"]["H"].item()  

        check = []
        for i in range(17):
            check = check + [True, True, False]

        while nr_frames_from_json_existing < frames_per_json_needed:
            frame_to_save = random.randint(0, trv-1)
            #print("frame selected: ", frame_to_save)
            kp = np.array(json_data[frame_to_save]["keypoints"])
            
            key_points = kp[check]
            key_points = key_points.reshape(-1, 2)

            image = np.zeros((H, W, 3), dtype=np.uint8)
            # Set all pixels to black
            image[:, :, :] = [0, 0, 0]  
            plt.imshow(image) # black background

            # Draw Posture
            for connection in connections:
                start_idx, end_idx = connection
                start_point = key_points[start_idx]
                end_point = key_points[end_idx]
                plt.plot([start_point[0], end_point[0]], [start_point[1], end_point[1]], color='red', linewidth=2)
            
            plt.ioff()
            plt.tight_layout()
            
            name_pic = clean_file_name + "_" + str(nr_frames_from_json_existing)
            plt.savefig(os.path.join(path_to_folder_containing_images, name_pic), transparent=True)
            nr_frames_from_json_existing += 1
            plt.clf()
    except IOError:
        pass
    except ValueError:
        pass

In [ ]:
not_done_yet = True
measure = pd.read_csv("/content/drive/Shareddrives/CV_videos/" + "other_docs/SVW.csv", sep = ";")[["FileName", "W", "H"]]

while not_done_yet:
    #try:
        for folder_sport in os.listdir(path_to_clean_json):
            if not os.path.isdir(os.path.join(path_to_images_from_json, folder_sport)):
                os.mkdir(os.path.join(path_to_images_from_json, folder_sport))

            print("Doing " + str(folder_sport) + "...")
            for cleaned_json_file in os.listdir(os.path.join(path_to_clean_json, folder_sport)):
                cjf = cleaned_json_file.strip(".json")
                path_to_dir_sport = os.path.join(path_to_images_from_json, folder_sport)
                nr_frames_from_clean_json_existing = len(glob.glob1(path_to_dir_sport, str(cjf) + "*"))

                if nr_frames_from_clean_json_existing < frames_per_clean_json_needed:
                    print(cjf)
                    create_posture_image_from_json(cjf, os.path.join(path_to_clean_json, folder_sport), os.path.join(path_to_images_from_json, folder_sport), nr_frames_from_clean_json_existing, frames_per_clean_json_needed, measure)
                else:
                    pass
        
        not_done_yet = False
    # except Exception as e:
    #     print("Restarting this cell")
    #     time.sleep(3) # wait for 10 seconds before restarting the cell
    #     IPython.display.display(IPython.display.HTML("<script>Jupyter.notebook.execute_cells_below()</script>"))

print("Done!")

Doing baseball...
Doing archery...
Doing gymnastics...
Doing basketball...
1246___c815d4e6e04f49119893be957b94bd1e
6160___03bad4b70ab14891a48ae2075d48776c
Doing bowling...
Doing discusthrow...
30716___50269a24aec34dde8694db8d91d354fa
Doing golf...
1136___fb458b551749422db714b4164248565d
Doing hammerthrow...
3092___327bc75e7bb24c1297ddbc303c2779fe
Doing highjump...
5244___b71539e3331c43e8baee70e831fcce8a
21553___dc580250010249f9a64769d3aced4289
Doing javelin...
20___2932ac6d65eb439eaaec362f17f6ac70
Doing longjump...
26354___09ed3a6fa112437cad5dbbea176801fc
Doing polevault...
891___82a7fdea847d4fd597833125279d4d53
Doing running...
Doing shotput...
20937___c1007e5445e246678e93b976b0433169
Doing weight...
Done!


<Figure size 640x480 with 0 Axes>

#---------------------------------------------------------------------------------------------------------------
# Move images into right shape for finetuner 
#---------------------------------------------------------------------------------------------------------------

# Move images from videos into correct folder for finetune

In [6]:
# !mkdir /content/drive/Shareddrives/CV_videos/Images_from_videos/analysis_split

data_split = pd.read_csv('/content/drive/Shareddrives/CV_videos/data_split.csv')
path_to_images_from_video = "/content/drive/Shareddrives/CV_videos/Images_from_videos/"
target_dir = "/content/drive/Shareddrives/CV_videos/Images_from_videos/analysis_split"


# create three directories
sub_dirs = ["train", "val", "test"]
for image_set in sub_dirs:
    if not os.path.isdir(os.path.join(target_dir, image_set)):
        os.mkdir(os.path.join(target_dir, image_set))

# create subdirectory for each with the sports images
for image_set in sub_dirs:
    for sport in os.listdir(path_to_images_from_video):
        if not os.path.isdir(os.path.join(target_dir, image_set, sport)):
            if sport != "analysis_split":
                os.mkdir(os.path.join(target_dir, image_set, sport))


# go file by file and move into correct directory
for sport in os.listdir(path_to_images_from_video):
    print(sport)
    if sport != "analysis_split":
        path_into_sport_dir = os.path.join(path_to_images_from_video, sport)
        for image in os.listdir(path_into_sport_dir):
            try: 
                im_name = re.sub(r"_[^_]*$", ".png", image)
                vid_name = im_name.rstrip(".png") + str(".mp4")
                where_to_move = data_split[data_split["FileName"] == vid_name]["SPLIT"].item()
                dest = os.path.join("/content/drive/Shareddrives/CV_videos/Images_from_videos/analysis_split", where_to_move, sport, image)
                src = os.path.join("/content/drive/Shareddrives/CV_videos/Images_from_videos/", sport, image)
            
                if not os.path.isfile(dest):
                    #print("not yet")
                    shutil.copyfile(src, dest)
            except ValueError:
                # occurs at .ipynb_checkpoints. Not sure where this file is coming from
                pass

        # check here and start moving 


baseball
gymnastics
basketball
bowling
discusthrow
golf
hammerthrow
highjump
javelin
longjump
polevault
running
shotput
weight
.ipynb_checkpoints
analysis_split
archery


# Move images from json into correct folder for finetuning

In [8]:
# !mkdir /content/drive/Shareddrives/CV_videos/Images_from_json/analysis_split
data_split = pd.read_csv('/content/drive/Shareddrives/CV_videos/data_split.csv')
path_to_images_from_json = "/content/drive/Shareddrives/CV_videos/Images_from_json/"
target_dir = "/content/drive/Shareddrives/CV_videos/Images_from_json/analysis_split"


# create three directories
sub_dirs = ["train", "val", "test"]
for image_set in sub_dirs:
    if not os.path.isdir(os.path.join(target_dir, image_set)):
        os.mkdir(os.path.join(target_dir, image_set))

# create subdirectory for each with the sports images
for image_set in sub_dirs:
    for sport in os.listdir(path_to_images_from_json):
        if not os.path.isdir(os.path.join(target_dir, image_set, sport)):
            if sport != "analysis_split":
                os.mkdir(os.path.join(target_dir, image_set, sport))


# go file by file and move into correct directory
for sport in os.listdir(path_to_images_from_json):
    print(sport)
    if sport != "analysis_split":
        path_into_sport_dir = os.path.join(path_to_images_from_json, sport)
        for image in os.listdir(path_into_sport_dir):
            try: 
                im_name = re.sub(r"_[^_]*$", ".png", image)
                vid_name = im_name.rstrip(".png") + str(".mp4")
                where_to_move = data_split[data_split["FileName"] == vid_name]["SPLIT"].item()
                dest = os.path.join("/content/drive/Shareddrives/CV_videos/Images_from_json/analysis_split", where_to_move, sport, image)
                src = os.path.join("/content/drive/Shareddrives/CV_videos/Images_from_json/", sport, image)
            
                if not os.path.isfile(dest):
                    #print("not yet")
                    shutil.copyfile(src, dest)
            except ValueError:
                # occurs at .ipynb_checkpoints. Not sure where this file is coming from
                pass

        # check here and start moving 


baseball
gymnastics
basketball
bowling
discusthrow
golf
hammerthrow
highjump
javelin
longjump
polevault
running
shotput
weight
analysis_split
